### Let, where

In [21]:
f x = x * x 
    where x = 7
f 10

49

In [28]:
let n = 10 in
    let ns = [0..n] in
        let nsquares = [x^2 | x <- ns] in
            sum nsquares

385

In [ ]:
(let y = 4 in 3 + y)*(let y = 2 in y*y*y)

In [ ]:
calcBmis xs = [bmi | (w, h) <- xs, let bmi = w / h ^ 2]

In [ ]:
countPositivesRec [] = 0
countPositivesRec (x:xs) = positive x + countPositivesRec xs
    where positive x = if x > 0 then 1 else 0

### Linking lists, pattern matching, multiline definitions, guards

In [ ]:
import Test.QuickCheck
import Data.Char

lookUp :: Char -> [(Char, Char)] -> Char
lookUp ch xs = head ([y | (x,y) <- xs, x == ch] ++ [ch])

lookUpRec :: Char -> [(Char, Char)] -> Char
lookUpRec ch [] = ch
lookUpRec ch ((a,b):xs) |
    a == ch = b |
    otherwise = lookUpRec ch xs

prop_lookUp :: Char -> [(Char, Char)] -> Bool 
prop_lookUp c k = lookUpRec c k == lookUp c k

quickCheck (\ c k -> collect (isAlpha c) (prop_lookUp))
-- /show

In [ ]:
['a'] /= ['b']

### Select

In [ ]:
"words"!!3
[11..]!!6

### Take and drop

In [ ]:
splitEachFive :: String -> [String]
splitEachFive s |
    s == [] = [] |
    length s < 5 = (splitEachFive (s ++ ['#'])) |
    otherwise = [(take 5 s)] ++ splitEachFive (drop 5 s)
    
splitEachFive "asdf23432fsaadfadsfd"

In [ ]:
prop_splitEachFive :: String -> Bool
prop_splitEachFive s = length (splitEachFive s) ==  (length s + 4) `div` 5

quickCheck prop_splitEachFive

### Take and concat

In [ ]:
prop2_splitEachFive :: String -> Bool
prop2_splitEachFive s = concat (splitEachFive s) == take (((length s + 4) `div` 5)*5) (s ++ "####")

quickCheck prop2_splitEachFive

### search

In [ ]:
search :: Eq a => [a] -> a -> [Int]
search xs y = [ i | (i,x) <- zip [0..] xs, x==y ]

searchRec :: Eq a => [a] -> a -> [Int]
searchRec xs y  =  srch xs y 0
  where
  srch :: Eq a => [a] -> a -> Int -> [Int]
  srch [] y i      =  []
  srch (x:xs) y i
    | x == y       =  i : srch xs y (i+1)
    | otherwise    =  srch xs y (i+1)

### zipHarsh

In [ ]:
zipHarsh :: [a] -> [b] -> [(a,b)]
zipHarsh [] []  =  []
zipHarsh (x:xs) (y:ys)  =  (x,y) : zipHarsh xs ys

dot :: Num a => [a] -> [a] -> a
dot xs ys  =  sum [ x*y | (x,y) <- zipHarsh xs ys ]

dotRec :: Num a => [a] -> [a] -> a
dotRec [] []          =  0
dotRec (x:xs) (y:ys)  =  x*y + dotRec xs ys

prop_dot :: Eq a => Num a => [a] -> Bool
prop_dot xs = dot xs xs == dotRec xs xs
quickCheck prop_dot

### Types

In [ ]:
type Para a b = (a, b)
pairsum :: Para Int Int -> Para Int Int -> (Para Int Int, Int)
pairsum para@(a,b) (c,d) = (para, a+b+c+d)

## DFAs

In [ ]:
import Test.QuickCheck.Modifiers -- such as Positive and NonNegative
import Test.QuickCheck

type Next state symbol =         -- transition function for a DFA
  state -> symbol -> state
type DFA state symbol = (        -- names as in tex Mathematical Methods for Linguistics
    [state]            -- k 
  , [symbol]           -- sigma 
  ,  state             -- q0 
  , Next state symbol  -- delta 
  , state -> Bool      -- f (accepting?)
  )

In [ ]:
final :: DFA st sy ->  st -> [sy] -> st -- given dfa initial state and inputs
final dfa                 s  []    = s  -- where do we end up
final dfa@(_,_,_,delta,_) s (x:xs) = final dfa (delta s x) xs

In [ ]:
accept :: DFA state symbol -> [symbol] -> Bool -- starting from start state do
accept dfa@(_, _, q0, _, f) xs = f (final dfa q0 xs) -- we reach an accepting state

### DFA for $\mathbb{Z_m}$

In [ ]:
basebmodm :: Int -> Int        -- modulus m and base b
  -> DFA Int Int               -- accepts strings base b encoding 
basebmodm m b = (              -- a multiple of m
    [0..(m-1)]  -- states
  , [0..(b-1)]  -- symbols 
  , 0           -- start at 0 
  , delta       -- see below 
  , (==0)       -- accepts 0
  ) where
  delta st sy = 
    (b*st + sy) `mod` m

### Product DFA

In [ ]:
prodDFA :: Eq sy => DFA st sy -> DFA st' sy -> DFA (st, st') sy
prodDFA (k,  sigma,  q0,  delta,  f)      -- the product of two DFA
        (k', sigma', q0', delta', f') =   -- recognises strings accepted
  if sigma /= sigma' then undefined else  -- by both of them
    (
      [(x,x') | x <- k, x' <- k']
    , sigma
    , (q0, q0')
    , delta''
    , f''
    )where
      delta'' (x, x') s = (delta x s, delta' x' s)
      f'' (q, q') = f q && f' q'

## Map

In [ ]:
squares = map sqr
    where
    sqr x = x * x
squares [-1, 2, 3]

## Filter

In [ ]:
filter (==7) [3,7,1,5,6,2,7,7,7,7,3,34,2,7]

## Fold

### Foldr

In [ ]:
foldr (\x y -> x + y + 1) 0 [1..10]
foldr (-) 10 [1..4] -- = (1 - (2 - (3 - (4 - 10)))) = 1 - 2 + 3 - 4 + 10 = 8

### Foldl

In [ ]:
foldl (-) 10 [1..4] -- = (((10 - 1) - 2) - 3) - 4 = 10 - 1 - 2 - 3 - 4 = 10-10 = 0

### Foldr1, Foldl1

In [ ]:
foldr (\x y -> x + y + 1) 0 [1..10]
foldr1 (\x y -> x + y + 1) [1..10]
foldl1 (\x y -> x + y + 1) [1..10]

In [ ]:
import Test.QuickCheck
foldProp :: [Int] -> Property
foldProp xs = (length xs > 0) ==> (foldl1 (max) xs) == (foldr1 (max) xs)
quickCheck foldProp

## Currying

In [ ]:
mul a b = a * b
times37 = mul 37
times37 1137

## Lambdas

In [ ]:
f_complicated :: [Int] -> Int
f_complicated xs  =  foldr (+) 0 (map sqr (filter pos xs))
    where
    sqr x  =  x*x
    pos x  =  x > 0

#### How to simplify this ?

In [ ]:
f_simpler :: [Int] -> Int
f_simpler xs  =  foldr (+) 0 (map (\x -> x * x) (filter (\x -> x > 0) xs))

In [ ]:
-- testing if f_complicated == f_simpler
import Test.QuickCheck
prop_f :: [Int] -> Bool
prop_f xs = f_complicated xs == f_simpler xs
quickCheck prop_f

The character \\ stands for $\lambda$

### $\beta$ Rule
$(\lambda x.N) M$  
**means**  
let $x = M$ in $N$

$(> 0)$ is shorthand for $(\x -> x > 0)$  
$(2*)$ is shorthand for $(\x -> 2*x)$  
__similarily__  
$(+ 1)$    
$(2 ˆ)$      
$(ˆ 2)$    

In [ ]:
f_simplest :: [Int] -> Int
f_simplest xs = foldr (+) 0 (map (^2) (filter (>0) xs))
prop_s :: [Int] -> Bool
prop_s xs = f_complicated xs == f_simplest xs
quickCheck prop_s

## Composition

In [ ]:
(.) :: (b -> c) -> (a -> b) -> (a -> c)
-- Ususal version
(.) f g x = f(g x)
-- Infix notation
(f . g) x  =  f (g x)

In [ ]:
((>0).(^2)) 0

# Thinking Functionally

##### Boring version

In [ ]:
f :: [Int] -> Int
f xs  =  foldr (+) 0 (map (^ 2) (filter (> 0) xs))

##### Examples of why we learn Haskell 
Lambda expressions and binding constructs

In [ ]:
f :: [Int] -> Int
f = foldr (+) 0 . map (^2) . filter (>0)

In [ ]:
(\f -> f 2) (\x -> ((\y -> x+y*y) (x+1)))

is the same as

In [ ]:
f2 = f 2 where
    f x = x*x + y where
        y = x + 1
f2

## Testing properties

In [ ]:
import Test.QuickCheck
prop_sum :: Int -> Property
prop_sum n  =  n >= 0 ==>  sum [1..n]  ==  (*) n (n+1) `div` 2
quickCheck prop_sum 

## Function application with `$`

### Definition
 `f $ x` = `f x`  
 `f $ g $ h x` = `f (g (h x))` 

### Precedence

function `($)` has the lowest precedence  
`f a b c` is the same as `((f a) b) c))`
while  
`f $ a b` is `f(a(b))`

In [ ]:
filter (\x -> x `mod` 2 == 1) $ map (^2) [1..10]
filter (\x -> x `mod` 2 == 1) . map (^2) $ [1..10]

## Finding a definition

In [ ]:
:d ($)
:d takeWhile

### Take while

In [ ]:
takeWhile (\x -> x `mod` 2 == 0 || x < 8) [1..]

### Implication, Bi-Implication, Reverse Implication
It seems that there are no built-in operators

## ZipWith, Uncurry

`zipWith::(a → b → c) → [a] → [b] → [c]`   
`zipWith` generalises `zip` by zipping with the function given as the first argument, instead of a tupling function. For example, `zipWith (+)` is applied to two lists to produce the list of corresponding sums

`uncurry ∷ (a → b → c) → ((a, b) → c`  
_uncurry converts a curried function to a function on pairs._

## Case of

In [9]:
head' :: [a] -> a  
head' xs = case xs of [] -> error "No head for empty lists!"  
                      (x:_) -> x  

In [11]:
case [3] of [] -> "Hurray"
            (x:xs) -> "HurrayHurray"
            _ -> "Sad"

"HurrayHurray"

## Elem, Replicate, dropWhile

In [ ]:
elem 3 [1,2,3]

In [ ]:
dropWhile (=='C') $ replicate 5 'C' ++ "A" ++ replicate 5 'T'

## Algebraic data types

In [ ]:
data List a  =  Nil
            |  Cons a (List a)
append :: List a -> List a -> List a
append Nil ys          =  ys
append (Cons x xs) ys  =  Cons x (append xs ys)

#### Algebraic data types with parameters (which are types)

In [ ]:
data Stree a = Tip a | Node (Stree a) a (Stree a)
etree = Node (Node (Node (Tip 3) 1 (Tip 4)) 3 (Node (Tip 2) 4 (Tip 2))) 5 (Node (Tip 2) 7 (Tip 2))

sumLeafs :: (Stree Int) -> Int
sumLeafs (Node a _ b) = sumLeafs a + sumLeafs b
sumLeafs (Tip x) = x

sumLeafs etree

## All and And

In [ ]:
all (==7) [7,7,7,7,7,1,7,7,7]
all (==7) [7,7,7,7,7]

In [ ]:
and [x `mod` 2 == 0 | x <- [1..100]]
and [(x*2+1) `mod` 2 == 1 | x <- [1..100]]

## Dot product

In [ ]:
dot :: Num a => [a] -> [a] -> a
dot = curry $ sum . (uncurry (zipWith (*)))

## Expression trees
### Evaluating expressions

In [ ]:
data Exp  =  Lit Int
          |  Add Exp Exp
          |  Mul Exp Exp

evalExp :: Exp -> Int
evalExp (Lit n)    =  n
evalExp (Add e f)  =  evalExp e + evalExp f
evalExp (Mul e f)  =  evalExp e * evalExp f

showExp :: Exp -> String
showExp (Lit n)    =  show n
showExp (Add e f)  =  par (showExp e ++ "+" ++ showExp f)
showExp (Mul e f)  =  par (showExp e ++ "*" ++ showExp f)

par :: String -> String
par s  =  "(" ++ s ++ ")"

In [ ]:
e0, e1 :: Exp
e0 = Add (Lit 2) (Mul (Lit 3) (Lit 3))
e0_infix = (Lit 2) `Add` ((Lit 3) `Mul` (Lit 3))
e1 = Mul (Add (Lit 2) (Lit 3)) (Lit 3)

test_Exp :: Bool
test_Exp =
      showExp e0 == "(2+(3*3))"
  &&  evalExp e0 == 11
  &&  showExp e1 == "((2+3)*3)"
  &&  evalExp e1 == 15
  &&  showExp e0_infix == showExp e0
  &&  evalExp e0_infix == evalExp e0
  
test_Exp

### Subsets

In [ ]:
subs :: [a] -> [[a]]
subs []      =  [[]]
subs (x:xs)  =  subs xs ++ [ x:ys | ys <- subs xs ]
subs (x:xs)  =  subs xs ++ map (x:) (subs xs)

subs "abcd"

### Using colon infix notation

In [ ]:
data Exp  =  Lit Int
          |  Exp :+: Exp
          |  Exp :*: Exp

evalExp :: Exp -> Int
evalExp (Lit n)    =  n
evalExp (e :+: f)  =  evalExp e + evalExp f
evalExp (e :*: f)  =  evalExp e * evalExp f

evalExp $ Lit 2 :+: Lit 3
evalExp $ Lit 2 :*: Lit 3

-- Just the colon at the beginning is essential

### Logical propositions

In [ ]:
type Name = String

-- using Prp rather than Prop to avoid conflict with Prop in QuickCheck
data Prp = Var Name
          | F
          | T
          | Not Prp
          | Prp :|: Prp
          | Prp :&: Prp
          deriving (Eq, Ord, Show)

type Names = [Name]
type Env = [(Name,Bool)]

## Removing duplicates

In [29]:
import Data.List
nub [1,2,3,4,1,2,12]

[1,2,3,4,12]

## Kinds and some type-foo

In [14]:
sum [1..10]

55

In [26]:
print_sth :: (Either Int String) -> String
print_sth (Left x) = show x
print_sth (Right x) = x

print_sth (Left 7)
print_sth (Right "abc")

do_sth :: (Either Int String) -> Int
do_sth (Left x) = x+1
do_sth (Right x) = (length x) + 1

do_sth $ Left 5
do_sth $ Right "abc"

:t Left

"7"

"abc"

6

4

Left :: forall a b. a -> Either a b